In [1]:
import pandas as pd
import numpy as np

In [9]:
result = pd.read_csv("./retrieval_results_heuristic_k5.csv")

In [10]:
result

,question_id,question_text,document_idx,embedding_method
0,646,Is the smallest penguin species the Little Blu...,"953,1182,1192,1204,2595",bert-base-uncased
1,646,Is the smallest penguin species the Little Blu...,"952,953,967,975,979",multi-qa-mpnet-base-dot-v1
2,646,Is the smallest penguin species the Little Blu...,"950,2467,2471,2596,2642",hkunlp-instructor-large
3,646,Is the smallest penguin species the Little Blu...,"951,952,953,979,981",intfloat-e5-small-v2
4,873,What does a citizen use to propose changes to ...,"25,656,657,658,2141",bert-base-uncased
...,...,...,...,...
395,794,Are turtles pets?,"2684,2688,2692,2693,2698",intfloat-e5-small-v2
396,332,Is he buried in the Princeton Cemetery of the ...,"525,699,706,1396,2226",bert-base-uncased
397,332,Is he buried in the Princeton Cemetery of the ...,"147,525,567,1967,2999",multi-qa-mpnet-base-dot-v1
398,332,Is he buried in the Princeton Cemetery of the ...,"117,289,567,2718,2903",hkunlp-instructor-large


In [14]:
from itertools import combinations

# --- helper: turn "953,1182,1192,1204,2595" into a set of ints ---
def parse_doc_idx(s):
    return set(map(int, str(s).split(',')))


# --- function to compute avg Jaccard distance inside one question group ---
def avg_jaccard_distance_per_question(group: pd.DataFrame) -> float:
    sets = group['document_idx'].apply(parse_doc_idx).tolist()
    
    # if fewer than 2 methods, variation is 0 by convention
    if len(sets) < 2:
        return 0.0
    
    dists = []
    for A, B in combinations(sets, 2):
        inter = len(A & B)
        union = len(A | B)
        jaccard_sim = inter / union if union > 0 else 0.0
        dists.append(1.0 - jaccard_sim)  # Jaccard distance
    
    return sum(dists) / len(dists)


# --- 1) average variation PER QUESTION ---
per_question_variation = (
    result.groupby('question_id')
      .apply(avg_jaccard_distance_per_question)
      .rename('avg_docidx_variation')
)

# --- 2) overall average variation across all questions (optional) ---
overall_avg_variation = per_question_variation.mean()
print("Overall avg variation across questions:", overall_avg_variation)


Overall avg variation across questions: 0.8935052910052909


/var/folders/3f/f3yfgyxj3csccqmb_dyksc1w0000gn/T/ipykernel_95564/1358113608.py:29: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(avg_jaccard_distance_per_question)


In [15]:
eval = pd.read_csv("result/rag_evaluation_summary.csv")

In [11]:
pd.concat([result,eval],axis=1).groupby('embedding_method').agg({'f1_score':['mean','std'],'cosine_similarity':['mean','std'],'bertscore_f1':['mean','std']}).reset_index()

embedding_method  f1_score           cosine_similarity            \
                                   mean       std              mean       std   
0           bert-base-uncased  0.217856  0.380715          0.653507  0.288320   
1     hkunlp-instructor-large  0.324301  0.439535          0.674187  0.293843   
2        intfloat-e5-small-v2  0.277485  0.413854          0.659634  0.292439   
3  multi-qa-mpnet-base-dot-v1  0.332001  0.437982          0.694386  0.287651   

  bertscore_f1            
          mean       std  
0     0.739820  0.120993  
1     0.771500  0.133702  
2     0.749955  0.120672  
3     0.770692  0.128890